In [1]:
from googleapiclient.discovery import build # для получения информации по API
import petl as etl # для загрузки и обработки данных
import pandas as pd # для выгрузки таблицы в postgresql
import sqlalchemy # для создания подключения к базе данных
import re # регулярные выражения

ModuleNotFoundError: No module named 'googleapiclient'

In [ ]:
youTubeApiKey = 'BIrandomsimbolsandnothingmoredudes_4bvOtzFsA' # API из аккаунта google 
youtube = build('youtube', 'v3', developerKey=youTubeApiKey) # Создание объекта, используя YouTube Data API v3

In [ ]:
channel_id = 'UC8LNafdLZNKqs3lbV5xAtNA' # id канала
content = youtube.channels().list(part='contentDetails', id = channel_id).execute() # получение детальной информации о канале

In [ ]:
playlistId = content['items'][0]['contentDetails']['relatedPlaylists']['uploads'] # id плейлиста загруженных видео

In [ ]:
allvideos = [] # список с видео

In [ ]:
nextPage_token = None

In [ ]:
res = youtube.playlistItems().list(playlistId = playlistId, maxResults = 100, part = 'snippet', pageToken = nextPage_token).execute()
allvideos += res['items']
nextPage_token = res.get('nextPageToken')

In [ ]:
# добавление оставшихся видео в список
while nextPage_token is not None:
    res = youtube.playlistItems().list(playlistId = playlistId, maxResults = 100, part = 'snippet', pageToken = nextPage_token).execute()
    allvideos += res['items']
    nextPage_token = res.get('nextPageToken')

In [ ]:
video_ids = list(map(lambda x: x['snippet']['resourceId']['videoId'], allvideos)) # получение id видео

In [ ]:
# получение статистики видеозаписей по их id
stats = []
for i in range(0, len(video_ids), 40):
    res = youtube.videos().list(id=','.join(video_ids[i:i+40]), part='statistics').execute()
    stats += res['items']

In [ ]:
channel_stats = youtube.channels().list(part='statistics', id = channel_id).execute() # получение статистики канала

In [ ]:
subscriberCount = channel_stats['items'][0]['statistics']['subscriberCount'] # количество подписчиков

In [ ]:
def links_remove(link):
    pattern = r'http\S+'
    match = re.findall(pattern, link)
    for m in match:
        url = m[:8]
        link = link.replace(url, '')
    return link

In [ ]:
data = []
# создание словаря со статистикой
for _videos, _stats in zip(allvideos, stats):
    row = {
        'title': _videos['snippet']['title'],
        'videoIDs': _videos['snippet']['resourceId']['videoId'],
        'video_description': links_remove(_videos['snippet']['description']),
        'publishedDate': _videos['snippet']['publishedAt'],
        'liked': _stats['statistics']['likeCount'],
        'disliked': _stats['statistics']['dislikeCount'],
        'views': _stats['statistics']['viewCount'],
        'comment': _stats['statistics']['commentCount']
    }
    data.append(row)

In [ ]:
table = etl.fromdicts(data) # создание petl таблицы

In [ ]:
table = etl.addfield(table,'subscriberCount', subscriberCount)

In [ ]:
table.toxlsx('YT_statistics.xlsx', mode='overwrite') # экспорт в Excel файл

In [ ]:
engine = sqlalchemy.create_engine('postgresql://{user}:{user_password}@{url}:{port}/{database_name}') # подключение к базе данных

In [ ]:
df = pd.DataFrame(columns=table[0], data=table[1:]) # создание DataFrame из petl-таблицы

In [ ]:
# экспорт в базу данных
df.to_sql('yt_analytics', engine, index=False, if_exists='replace', dtype={
    'title': sqlalchemy.VARCHAR(100),
    'videoIDs': sqlalchemy.VARCHAR(50),
    'video_description': sqlalchemy.Text(),
    'publishedDate': sqlalchemy.Date(),
    'liked': sqlalchemy.Integer(),
    'disliked': sqlalchemy.Integer(),
    'views': sqlalchemy.Integer(),
    'comment': sqlalchemy.Integer(),
    'subscriberCount': sqlalchemy.Integer()
})